In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
import re

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
dataset = load_dataset('ag_news')
train_data = dataset['train']
test_data = dataset['test']

In [4]:
def simple_tokenizer(text):
    text = text.lower()
    tokens = re.findall(r'\b\w+\b', text)
    return tokens

In [5]:
counter = Counter()
for example in train_data:
    counter.update(simple_tokenizer(example['text']))

specials = ['<pad>', '<unk>']
vocab = {token: idx for idx, token in enumerate(specials + list(counter.keys()))}
vocab['<pad>'] = 0
vocab['<unk>'] = 1
inv_vocab = {idx: token for token, idx in vocab.items()}

vocab_size = len(vocab)

In [6]:
def encode(text):
    tokens = simple_tokenizer(text)
    return [vocab.get(token, vocab['<unk>']) for token in tokens]

def collate_batch(batch):
    labels = torch.tensor([item['label'] for item in batch], dtype=torch.long)
    text_list = [torch.tensor(encode(item['text']), dtype=torch.long) for item in batch]
    texts = pad_sequence(text_list, batch_first=True, padding_value=vocab['<pad>'])
    return texts, labels

In [7]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False, collate_fn=collate_batch)

In [ ]:
class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, num_classes, max_len=512):
        super().__init__()
        self.token_embed = nn.Embedding(vocab_size, embed_dim)
        self.pos_embed = nn.Embedding(max_len, embed_dim)

        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        batch_size, seq_len = x.shape
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0).expand(batch_size, seq_len)
        x = self.token_embed(x) + self.pos_embed(positions)

        x = x.permute(1, 0, 2)  # (seq_len, batch_size, embed_dim)
        x = self.transformer_encoder(x)
        x = x.permute(1, 0, 2)  # (batch_size, seq_len, embed_dim)

        x = x.mean(dim=1)  # mean pooling
        return self.fc(x)

In [9]:
embed_dim = 128
num_heads = 4
num_layers = 2
num_classes = 4

model = TransformerClassifier(vocab_size, embed_dim, num_heads, num_layers, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

epochs = 5

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:375: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [10]:
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for texts, labels in train_loader:
        texts, labels = texts.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    print(f"Epoch {epoch+1}: Loss={total_loss/len(train_loader):.4f}, Train Acc={correct/total:.4f}")

Epoch 1: Loss=0.5289, Train Acc=0.7934
Epoch 2: Loss=0.2734, Train Acc=0.9081
Epoch 3: Loss=0.2147, Train Acc=0.9279
Epoch 4: Loss=0.1773, Train Acc=0.9414
Epoch 5: Loss=0.1491, Train Acc=0.9509


In [11]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for texts, labels in test_loader:
        texts, labels = texts.to(device), labels.to(device)
        outputs = model(texts)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {correct/total:.4f}")

Test Accuracy: 0.9041
